In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_val_score
from tqdm import tqdm


In [3]:

from sklearn.ensemble import RandomForestRegressor

Loading data



In [4]:
train_bike = pd.read_csv("train.csv")

#test_bike = pd.read_csv("test.csv")

In [5]:
train_bike["datetime"].head()

0    2011-01-01 00:00:00
1    2011-01-01 01:00:00
2    2011-01-01 02:00:00
3    2011-01-01 03:00:00
4    2011-01-01 04:00:00
Name: datetime, dtype: object

In [6]:
train_bike["datetime"] = pd.to_datetime(train_bike["datetime"])

train_bike["datetime"].describe()

count                   10886
unique                  10886
top       2011-06-09 04:00:00
freq                        1
first     2011-01-01 00:00:00
last      2012-12-19 23:00:00
Name: datetime, dtype: object

In [7]:
train_bike["dayofweek"] = train_bike["datetime"].dt.dayofweek      #The day of the week with Monday=0, Sunday=6
train_bike["year"] = train_bike["datetime"].dt.year
train_bike["month"] = train_bike["datetime"].dt.month
train_bike["day"] = train_bike["datetime"].dt.day
train_bike["hour"] = train_bike["datetime"].dt.hour

In [8]:
X = train_bike[["season", "holiday", "workingday", "weather","temp","humidity","windspeed"]]
y = train_bike["registered"]

In [9]:
from sklearn.model_selection import train_test_split #dzielnie na zetsawy
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33, random_state=42)

In [10]:
best_score = 0
parameters = {}

In [11]:
from tqdm import tqdm

In [ ]:
for n_estimators in tqdm(range(1,100)):
    for max_features in np.arange(0.05, 1.01, 0.05):
        for min_samples_split in range(2,21):
            for min_samples_leaf in range(1,21):
                for criterion in ["mse", "mae"]:
                    for bootstrap in [True, False]:
                        clf_rfr = RandomForestRegressor(max_features = max_features, min_samples_split = min_samples_split, 
                                                     min_samples_leaf = min_samples_leaf, criterion = criterion,
                                                      bootstrap = bootstrap )
                        try:
                            scores = cross_val_score(clf_rfr, X_train, y_train, cv = 5)
                        except ValueError as e:
                            #print(e)
                            continue
                        if scores.mean() > best_score:
                            #print("New best parameters!")
                            parameters["best_features"] = max_features
                            parameters["best_min_split"] = min_samples_split
                            parameters["best_min_leaf"] = min_samples_leaf
                            parameters["best_criterion"] = criterion
                            parameters["bootstrap"] = bootstrap
                            best_score = scores.mean()
                            print("Accuracy: {}".format(scores.mean()))
                            print("max_features: {}, min_samples_split: {}, min_samples_leaf: {}, criterion: {}, bootstrap: {}".format(max_features, 
                                                                                                                     min_samples_split, 
                                                                                                             min_samples_leaf, 
                                                                                                             criterion, bootstrap))

  0%|          | 0/99 [00:00<?, ?it/s]

Accuracy: 0.13129258880923098
max_features: 0.05, min_samples_split: 2, min_samples_leaf: 1, criterion: mse, bootstrap: True
Accuracy: 0.1908595864133889
max_features: 0.05, min_samples_split: 2, min_samples_leaf: 1, criterion: mae, bootstrap: True
Accuracy: 0.20890719253350398
max_features: 0.05, min_samples_split: 2, min_samples_leaf: 2, criterion: mse, bootstrap: True
Accuracy: 0.2262835909679536
max_features: 0.05, min_samples_split: 2, min_samples_leaf: 3, criterion: mse, bootstrap: True
Accuracy: 0.23545489391659102
max_features: 0.05, min_samples_split: 2, min_samples_leaf: 4, criterion: mse, bootstrap: True
Accuracy: 0.2432141438291449
max_features: 0.05, min_samples_split: 2, min_samples_leaf: 5, criterion: mse, bootstrap: False
Accuracy: 0.24402730790877653
max_features: 0.05, min_samples_split: 2, min_samples_leaf: 6, criterion: mse, bootstrap: False
Accuracy: 0.24474320518096532
max_features: 0.05, min_samples_split: 5, min_samples_leaf: 8, criterion: mse, bootstrap: False


In [ ]:
clf_rfr = RandomForestRegressor(criterion = 'mse')

In [ ]:
clf_rfr.fit(X = X_train, y = y_train)

In [ ]:
prediction_rfr = clf_rfr.predict(X_test)

In [ ]:
prediction_rfr